# TinyLlama Tool Calling Lab

## Introduction
Welcome to a practical tool calling laboratory using TinyLlama-1.1B! This lab demonstrates how to implement function calling with a lightweight local model that uses minimal resources.

### What You'll Learn
1. Set up TinyLlama-1.1B for local inference
2. Create and use custom tools (calculator, weather, text analysis)
3. Implement pattern-based tool calling
4. Handle tool execution and responses

### Why TinyLlama?
- **Lightweight**: Only ~1GB RAM required
- **Fast**: Quick inference on CPU
- **Local**: No API keys needed
- **Educational**: Perfect for learning tool calling concepts

### Prerequisites
- Python 3.9+
- ~2GB free disk space
- Internet connection for model download

## Step 1: Setup and Import Libraries

First, let's import all necessary libraries and check our environment.

In [ ]:
# Core libraries
import os
import sys
import json
import time
from typing import Dict, Any, List, Optional

# Add tools directory to path
sys.path.append('./tools')

# Import our custom tools
try:
    from custom_tools import (
        CalculatorTool, WeatherTool, FileOperationsTool, TextAnalysisTool,
        TOOL_SCHEMAS, execute_tool
    )
    from ollama_client import OllamaClient, format_tool_call_prompt, extract_tool_call
    print("✅ Custom tools imported successfully")
except ImportError as e:
    print(f"⚠️ Could not import custom tools: {e}")
    print("📝 Note: Some imports may fail if dependencies aren't installed yet")

# Check Python version
print(f"🐍 Python version: {sys.version}")
print(f"📁 Current working directory: {os.getcwd()}")

## Step 2: Environment Check

Let's check if we have the required libraries and automatically set up TinyLlama-1.1B:

In [ ]:
# Check if transformers is available
def check_transformers():
    try:
        import transformers
        import torch
        return True
    except ImportError:
        return False

# Simple environment check
transformers_available = check_transformers()

print("🔍 Environment Check:")
print(f"  Transformers available: {'✅' if transformers_available else '❌'}")

if transformers_available:
    print("✅ Ready to use TinyLlama with Transformers")
    USE_TRANSFORMERS = True
else:
    print("❌ Please install: pip install transformers torch")
    USE_TRANSFORMERS = False

## Step 3: Load TinyLlama Model

Now let's load TinyLlama-1.1B for tool calling:

In [ ]:
if not USE_TRANSFORMERS:
    print("❌ Transformers not available. Please install required packages.")
    MODEL_INITIALIZED = False
else:
    print("🚀 Loading TinyLlama-1.1B...")
    
    try:
        from transformers import AutoTokenizer, AutoModelForCausalLM
        import torch
        
        # Check for optimal device (MPS on Mac M1/M2)
        if torch.backends.mps.is_available():
            device = "mps"  # Metal Performance Shaders for Mac
            print("🚀 Using Metal Performance Shaders (MPS) for Mac M2")
        else:
            device = "cpu"
            print("🖥️ Using CPU for inference")
        
        # Load TinyLlama model
        model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        print(f"📦 Loading {model_name}...")
        
        # Optimize for Mac M2 - use float16 to reduce memory usage
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,  # Much more memory efficient
            low_cpu_mem_usage=True,
            device_map="auto"  # Automatic device placement
        )
        
        # Move model to optimal device
        model = model.to(device)
        print(f"🖥️ Model loaded on: {device}")
        print("🖥️ Using optimized settings for Mac M2")
        
        # Set padding token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        # Quick test
        print("🧪 Testing model...")
        test_input = tokenizer("Hello!", return_tensors="pt").to(device)
        with torch.no_grad():
            output = model.generate(**test_input, max_new_tokens=3, do_sample=False)
        
        MODEL_INITIALIZED = True
        print("✅ TinyLlama loaded successfully!")
        print(f"💾 Memory optimized for Mac M2 (using {device})")
        
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        MODEL_INITIALIZED = False

## Step 4: Define Tools and Functions

Let's explore the tools we've defined and understand how they work:

In [ ]:
# Display available tools
print("🛠️ Available Tools:")
print("=" * 50)

for tool_name, tool_schema in TOOL_SCHEMAS.items():
    func_info = tool_schema['function']
    print(f"\n📋 {func_info['name']}")
    print(f"   Description: {func_info['description']}")
    print(f"   Parameters: {list(func_info['parameters']['properties'].keys())}")

print("\n" + "=" * 50)

In [ ]:
# Test our tools manually
print("🧪 Testing Tools Manually:")
print("=" * 30)

# Test calculator
print("\n🔢 Calculator Test:")
result1 = execute_tool("calculator_add", {"a": 15, "b": 27})
print(f"  15 + 27 = {result1}")

result2 = execute_tool("calculator_multiply", {"a": 8, "b": 7})
print(f"  8 × 7 = {result2}")

# Test weather (mock)
print("\n🌤️ Weather Test:")
weather = execute_tool("get_weather", {"city": "Milan", "country": "IT"})
print(f"  Weather in Milan: {json.dumps(weather, indent=2)}")

# Test text analysis
print("\n📝 Text Analysis Test:")
sample_text = "Machine learning and artificial intelligence are transforming the way we work with data. Natural language processing enables computers to understand human language."
analysis = execute_tool("analyze_text", {"text": sample_text, "max_keywords": 5})
print(f"  Analysis results: {json.dumps(analysis, indent=2)}")

print("\n✅ All tools working correctly!")

## Step 5: Tool Calling Implementation

Now let's implement tool calling with TinyLlama:

In [ ]:
def process_user_request(user_message: str) -> str:
    """Process user request with pattern-based tool calling"""
    
    print(f"👤 User: {user_message}")
    
    if not MODEL_INITIALIZED:
        return "❌ Model not initialized."
    
    # Simple pattern matching for tool calling
    msg_lower = user_message.lower()
    
    # Check for calculations
    calc_keywords = ["multiply", "multiplied", "divide", "divided", "add", "plus", "calculate", "what is"]
    if any(keyword in msg_lower for keyword in calc_keywords):
        return handle_calculation(user_message)
    
    # Check for weather
    elif "weather" in msg_lower:
        return handle_weather(user_message)
    
    # Check for text analysis
    elif "analyz" in msg_lower and "text" in msg_lower:
        return handle_text_analysis(user_message)
    
    # Default responses
    else:
        return "I can help with calculations, weather queries, and text analysis. What would you like me to do?"

def handle_calculation(user_message: str) -> str:
    """Handle calculation requests"""
    import re
    numbers = re.findall(r'\d+', user_message)
    if len(numbers) >= 2:
        a, b = int(numbers[0]), int(numbers[1])
        if any(word in user_message.lower() for word in ["multiply", "multiplied", "*", "times"]):
            result = execute_tool("calculator_multiply", {"a": a, "b": b})
            return f"The result of {a} × {b} is {result}"
        elif any(word in user_message.lower() for word in ["divide", "divided", "/"]):
            result = execute_tool("calculator_divide", {"a": a, "b": b})
            return f"The result of {a} ÷ {b} is {result}"
        else:
            result = execute_tool("calculator_add", {"a": a, "b": b})
            return f"The result of {a} + {b} is {result}"
    return "Please provide two numbers for calculation."

def handle_weather(user_message: str) -> str:
    """Handle weather requests"""
    cities = ["Rome", "Milan", "Naples", "Florence"]
    for city in cities:
        if city.lower() in user_message.lower():
            result = execute_tool("get_weather", {"city": city, "country": "IT"})
            return f"Weather in {city}: {result['condition']}, {result['temperature']}°C"
    # Default to Rome
    result = execute_tool("get_weather", {"city": "Rome", "country": "IT"})
    return f"Weather in Rome: {result['condition']}, {result['temperature']}°C"

def handle_text_analysis(user_message: str) -> str:
    """Handle text analysis requests"""
    text_start = user_message.find(":") + 1
    if text_start > 0:
        text_to_analyze = user_message[text_start:].strip()
    else:
        text_to_analyze = user_message
    
    if len(text_to_analyze) < 10:
        return "Please provide more text to analyze."
    
    result = execute_tool("analyze_text", {"text": text_to_analyze, "max_keywords": 5})
    return f"Analysis complete! Found {result['word_count']} words and keywords: {', '.join(result['keywords'])}"

print("🎯 Tool calling system ready!")

## Step 6: Test Tool Calling

Let's test our tool calling system with different types of requests:

In [ ]:
# Example 1: Simple calculation
print("📊 Example 1: Calculation Request")
print("=" * 40)

response1 = process_user_request("What is 156 multiplied by 23?")
print(f"🤖 Response: {response1}")
print("\n")

In [ ]:
# Example 2: Weather query
print("🌤️ Example 2: Weather Request")
print("=" * 40)

response2 = process_user_request("What's the weather like in Rome?")
print(f"🤖 Response: {response2}")
print("\n")

In [ ]:
# Example 3: Text analysis
print("📝 Example 3: Text Analysis Request")
print("=" * 40)

text_to_analyze = """Artificial Intelligence is revolutionizing many industries. 
Machine learning algorithms can process vast amounts of data to identify patterns and make predictions. 
Deep learning, a subset of machine learning, uses neural networks to solve complex problems."""

response3 = process_user_request(f"Please analyze this text and extract the main keywords: {text_to_analyze}")
print(f"🤖 Response: {response3}")
print("\n")

In [ ]:
# Example 4: No tool needed
print("💬 Example 4: General Conversation")
print("=" * 40)

response4 = process_user_request("What are the benefits of using local LLMs?")
print(f"🤖 Response: {response4}")
print("\n")

## Step 8: Error Handling

Let's test how our system handles error conditions:

In [ ]:
def test_error_handling():
    """Test various error conditions"""
    print("🛡️ Testing Error Handling:")
    print("=" * 30)
    
    # Test 1: Invalid tool parameters
    print("\n1. Invalid calculator parameters:")
    try:
        result = execute_tool("calculator_divide", {"a": 10, "b": 0})
        print(f"   Result: {result}")
    except Exception as e:
        print(f"   ❌ Error (expected): {e}")
    
    # Test 2: Missing required parameters
    print("\n2. Missing required parameters:")
    try:
        result = execute_tool("calculator_add", {"a": 5})  # Missing 'b'
        print(f"   Result: {result}")
    except Exception as e:
        print(f"   ❌ Error (expected): {e}")
    
    # Test 3: Unknown tool
    print("\n3. Unknown tool:")
    try:
        result = execute_tool("unknown_tool", {})
        print(f"   Result: {result}")
    except Exception as e:
        print(f"   ❌ Error (expected): {e}")
    
    # Test 4: Invalid text analysis
    print("\n4. Empty text analysis:")
    try:
        result = execute_tool("analyze_text", {"text": ""})
        print(f"   Result: {result}")
    except Exception as e:
        print(f"   ❌ Error: {e}")
    
    print("\n✅ Error handling tests completed")

test_error_handling()

## Step 9: Interactive Testing

Test more queries by modifying the examples below:

In [ ]:
# Try your own queries here by modifying these examples:

test_queries = [
    "What is 25 times 8?",
    "What's the weather in Milan?", 
    "Analyze this text: TinyLlama is a small but powerful language model.",
    "Calculate 100 plus 50"
]

print("🧪 Testing custom queries:")
for i, query in enumerate(test_queries, 1):
    print(f"\n--- Test {i} ---")
    response = process_user_request(query)
    print(f"🤖 Response: {response}")

print("\n💡 Try changing the queries above to test different requests!")

## Conclusion

🎉 **Congratulations!** You've successfully implemented tool calling with TinyLlama!

### What You've Learned:

1. **Model Setup**: Loaded TinyLlama-1.1B locally
2. **Tool Integration**: Connected calculator, weather, and text analysis tools
3. **Pattern Matching**: Implemented request classification without complex parsing
4. **Tool Execution**: Handled tool calls and formatted responses

### Key Takeaways:
- **Simple Patterns Work**: Basic keyword matching can effectively route requests
- **Lightweight Models**: TinyLlama provides good functionality with minimal resources
- **Direct Tool Calling**: You don't always need complex prompt engineering

### Next Steps:
- Try adding more tools (file operations, web search, etc.)
- Experiment with different classification patterns
- Test with more complex queries

**Happy coding with TinyLlama!** 🚀